In [1]:
import db_conn
import pandas as pd
import re

/home/grace/.local/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
conn = db_conn.get_connection()

## 1. 추출된 전체 약물
추출된 전체 약물 중 식별이 가능한 UNIQUE한 약물만을 추려냈습니다.

In [3]:
def get_total_drugs():
    cur = conn.cursor()
#     cur.execute("""
#     SELECT DISTINCT(TRIM(d.tt_drug)) AS t_d
#     FROM
#     (SELECT id, tt_drug
#     FROM article_table_sentences_m M,
#         UNNEST(STRING_TO_ARRAY(t_drug, ' , ')) AS tt_drug
#     WHERE (tt_drug='') is false) as d;
#     """)

# topic num 지정됨
    cur.execute("""
    SELECT DISTINCT(TRIM(d.tt_drug)) AS t_d
    FROM
    (SELECT id, tt_drug
    FROM 
        (SELECT * 
        FROM article_table_sentences_m
        WHERE table_id IN 
        (SELECT id FROM article_tables WHERE final_topic IS NOT NULL)) M,
        UNNEST(STRING_TO_ARRAY(t_drug, ' , ')) AS tt_drug
    WHERE (tt_drug='') is false) as d;
    """)
    return cur.fetchall()

def get_drug_dictionary():
    cur = conn.cursor()
    cur.execute("""
    select distinct(val) from (
    (select distinct(lower(cui1_str)) as val from dict_collapsed_final)
    UNION
    (select distinct(lower(cui2_str)) as val from dict_collapsed_final)
    ) final;
    """)
    return cur.fetchall()

In [4]:
drugs = pd.DataFrame(get_total_drugs())

In [5]:
# drugs.to_csv('./drugs.csv', sep=',')

In [6]:
drugs.columns=['name']

In [7]:
drugs['refined'] = ''
drugs.head()

,name,refined
0,edoxaban,
1,rivastigmine,
2,levofloxacin-non-susceptible,
3,fluphenazine,
4,procarbazine,


#### 전처리 전 총 약물의 수는 1197개 입니다

In [8]:
drugs.shape

(1197, 2)

### 2. 전처리

A. dictionary를 참고하여, 정확히 일치하는 약물명은 전처리 없이 포함합니다.

In [9]:
dictionary = pd.DataFrame(get_drug_dictionary())
dictionary.columns = ['t_drug']

In [10]:
len(dictionary)

23856

In [11]:
dictionary[:10]

,t_drug
0,digoxin didier
1,efaproxiral
2,vitaminum a
3,urion
4,klonopin
5,fositens
6,horacort
7,org 5222
8,tenstaten
9,furosemidu


In [12]:
def apply_dictionary():
    for ind, dr in drugs[drugs['refined'] == ''].iterrows():
    #     print(ad['name'].strip())
        name = dr['name'].strip()
        exact_mathing = dictionary.loc[dictionary.t_drug==name]

        if len(exact_mathing)==0:
            if name[0] in ['"', '.', ':', '-', '(', ')']:
                name = name[1:]
            if name[len(name)-1] in ['"', '.', ':', '-', '(', ')']:
                name = name[:len(name)-1]
    #         print(name)

            exact_mathing = dictionary.loc[dictionary.t_drug==name]

        if len(exact_mathing)>0:
            drugs.loc[ind]['refined'] = name

apply_dictionary()

In [13]:
drugs[drugs['refined'] != '']

,name,refined
0,edoxaban,edoxaban
1,rivastigmine,rivastigmine
3,fluphenazine,fluphenazine
4,procarbazine,procarbazine
8,ixekizumab,ixekizumab
9,famotidine,famotidine
10,electrolytes,electrolytes
11,insulin lispro,insulin lispro
12,tryptophan,tryptophan
13,daclizumab,daclizumab


#### B. 약물을 포함하고 있는 구절이 추출 된 경우, 약물 외적인 단어들을 제거합니다

## 선생님께서 need_to_refined_hyj.xlsx파일을 통해 약물 이름이 아니라고 지정해주신 목록을 추가하였습니다.

In [14]:
replace_with_space = ['-long-acting', '-treated', '-producing', '-high', '-involved-field', '-based', '-studies', '-non-susceptible',\
                      '-susceptible', '-dependent', '-inducible', '-independent', '-eligible', '-rescue', '-sensitive', '-dense',\
                      'infarction-', '11c]-', 'log[', '[99mtc]', '-avid', '-subcutaneous', '-coated', '-activatable', '-total', '-corrected', \
                      '-predicted', '-gated', '-redo-surgery', '-group', '-assigned', '-allocated', '-reduced', '-combination', \
                      '-standard', '-associated', '-ischemic', '-resistant', '-naive', '-sparing', 'concentration', '[', ']',\
                      'low-molecular-weight-', '-extended', '-meter', '-stimulated', 'subtotal', '-labeled', '-matching',\
                      '-resistance;', '-maintenance', '-rate', '-facilitated', '-replacement', '-to-renin', '-gits', '-treated',\
                      '-right', '-or-2-h', '-treatment', '-boosting', '-resistance', '-continuation', '-rich', '&#8211',\
                      '-impregnated', '-dispensation', '-pemetrexed', '-correctable', '-carry-on', '-oral', '-intravenous', '-bare-metal', '-dose',\
                      '-equivalent', '-induced', '-positive', '-related', '-stimulating', '-confirmed', '-controlled', '(', '-level',\
                      '-releasing', '-reuptake', '-eluting', '-containing', '-binding', 'vaccines', 'vaccine', 'change', 'vsstandard',\
                      '-sham', '-adjusted', 'post-', '5m-', '20m-', '-provoked', '<3.2', '<3.3', '30-', '-synthase', '-responsive',\
                      '-pa-824', 'formulation-', 'mi-', 'discretion-', 'polychemotherapy', 'chemotherapy', 'seizures-','crack-',\
                      '-myocardial', 'exercise-', '—4803', '-low', 'mothers', 'von willebrand factor', 'others', '—1688', '-13',
                      # 약물 이름이 아닌 경우
                      'food-borne', 'fast-food', 'food-coping', 'grass-pea-food', 'food-systems', 'food-bolus', 'food-specific', \
                      'food-frequency', 'non-food', 'blood plasma', 'medical air', 'tree pollen', 'grass pollen', 
                      # 추가 약물이름이 아닌 경우
                      'post-food', 'tc-sulfur', 'iodine-to-creatinine', 'protein c2', 'tc–sulfur', 'blood-glucose', \
                      'cy5.5-arginine-glycine-aspartate', 'biotin-aacctgaccacagaggagaggc-3', 'n-2-mercaptopropionylglycine', \
                      'biotin-tccctcatccctgccatgt-3', 'animals-l-arg', 'biotin-cacggcacctggcctaaag-3', 'buserelin+cpax14',\
                      # 추가 확인 필요
                      'albumin:creatinine', 'glucose-insulin-potassium', 'oxygen-18', 'cyclooxygenase-2', '18f-fluorodeoxyglucose', 'cyclo-oxygenase'
                     ]

In [15]:
rep = dict((re.escape(k), ' ') for k in replace_with_space)
pattern = re.compile("|".join(rep.keys()))

for ind, row in drugs.iterrows():
    if row['refined']=='':
        refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
        if refined.strip() == '':
            drugs.drop(ind, inplace=True)
        elif row['name'].strip() != refined.strip():
            drugs.loc[ind]['refined'] = refined
            print(row['name'])
            print(drugs.loc[ind]['refined'])
        else:
            print(row['name'])
        print('===============================')

drugs = drugs.reset_index(drop=True)

levofloxacin-non-susceptible
levofloxacin 
epirubicin-p-cyclophosphamide
pseudo-cholinesterase
non-naproxen
sirolimus-eluting
sirolimus 
deutetrabenazine
capecitabine-cisplatin
isoniazidthiacetazone
sulfadoxine-pyrimethamine
heparinglycoprotein
rituximab-chop
nifurtimox-eflornithine
acarbose–metformin
crizotinib-treated
crizotinib 
amodiaquine+sp
lipoprotein cholesterol
acadesine-treated
acadesine 
amphotericin bparomomycin
spartesunate
prothrombin
oxygen-dependent
oxygen 
oxaliplatin–fluoropyrimidine–bevacizumab
mono-cyclophosphamide
gemcitabine-based
gemcitabine 
taxane-cyclophosphamide
vandetanibdocetaxel
epirubicin-cyclophosphamide
tetracyclines
ceftriaxone-resistant
ceftriaxone 
pa-824-pyrazinamide
erlotinib-based
erlotinib 
cyclophosphamide-cyclophosphamide
albumin<3.2
albumin 
folfox–cetuximab
isoniazid-rifampicin-pyrazinamide-ethambutol
thrombin-antithrombin
roflumilast-treated
roflumilast 
atenolol-eligible
atenolol 
bparomomycin
dl-sotalol
carbamazepine-controlled
carbamazepi

#### 전처리와 약물이 아닌 레코드를 제거하면 1189개가 남습니다

In [16]:
drugs.shape

(1189, 2)

## 복합 약물 정제

### 선생님께서 주신 엑셀 파일을 기반으로 정제합니다

In [17]:
replace_excel = pd.read_excel('./need_to_refined_hyj.xlsx', header=None)
replace_excel.columns = ['id', 'key', 'replace_with', '-', '-', '-']
replace_excel = replace_excel[['key', 'replace_with']]
replace_excel.drop_duplicates(inplace=True)
replace_excel.head()

,key,replace_with
0,17-beta-estradiol,17-beta-estradiol
1,17-beta-estradiol-sham,17-beta-estradiol
2,17-hydroxyprogesterone,17-hydroxyprogesterone
3,5-flucytosine,5-flucytosine
4,5-hydroxyindoleacetic acid,5-hydroxyindoleacetic acid


In [18]:
replace_excel.tail()

,key,replace_with
392,zidovudine-lamivudine-nelfinavir,zidovudine + lamivudine + nelfinavir
393,azt+3tc+abacavir+efv,zidovudine lamivudine abacavir efavirenz
394,azt3tcamprenavir,zidovudine lamivudine amprenavir
395,5-fluorouracil,5-fluorouracil
396,methionine-threonine,methionine-threonine


In [19]:
replace_excel.shape

(393, 2)

In [20]:
def replace_with_manual_curation(drugs):
    for ind, row in replace_excel.iterrows():
        for d_ind, d_row in drugs.loc[drugs.name.str.strip()==row['key']].iterrows():
            origin_str = d_row['name'].replace(row['key'], row['replace_with'])

            drugs.loc[d_ind]['refined'] = origin_str 
    return drugs

drugs = replace_with_manual_curation(drugs)

In [21]:
drugs[drugs.refined.str.find('17-beta-estradiol')>=0]

,name,refined
1187,17-beta-estradiol,17-beta-estradiol


In [22]:
drugs[drugs.refined.str.find('zidovudine lamivudine amprenavir')>=0]

,name,refined
255,azt3tcamprenavir,zidovudine lamivudine amprenavir


## 아직 추가 전처리가 필요한 약물 수

In [23]:
drugs[drugs['refined'] == ''].shape

(97, 2)

#### B. 약-부작용 관계를 나타는 것이 아닌 경우, 지웁니다

### 선생님께서 피드백 주신데로 non은 제외하였습니다.
### 선생님께서 피드백 주신데로 약물명 전처리에서 collagen, blood, glucose, oxygen, nitrogen-13 등을 추가하였습니다.

In [24]:
non_drug_ad_rel = ['pseudo', 'placebo', 'negative', 'free', 'withdrawal', 'blocking', 'deficient', 'inhibiting',
                  '-prev', '-insensitive', 'resistant', 'refractory', 'pre-', 'lipoprotein', '-untreated', 'anti-',
                  '-receptor', '-tolerance', '-intolerant', 'non-secretin', 'collagen', 'blood', 'glucose', 'oxygen',\
                  'nitrogen-13', 'thrombin-antithrombin', 'fetoprotein', 'renin', 'cholesterol', 'albumin:creatinine']

rep = dict((re.escape(k), ' ') for k in non_drug_ad_rel)
pattern = re.compile("|".join(rep.keys()))

In [25]:
for ind, row in drugs.iterrows():
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if row['name'].strip() != refined.strip():
        print(row['name'])
        drugs.drop(ind, inplace=True)
        print('===============================')

drugs = drugs.reset_index(drop=True)

pseudo-cholinesterase
lipoprotein cholesterol
oxygen-dependent
ceftriaxone-resistant
thrombin-antithrombin
interferon-intolerant
glucose
salmeterolplacebo
collagen
oxygenation
b-glucose
placebodocetaxel
anti-exenatide
progesterone-receptor
18f-fluoro-2-deoxy-d-glucose
glucose]
oxygen
glucose-lowering
anti-evolocumab
ipoprotein cholesterol
sodium-glucose
pre-carvedilol
glucose-confirmed
procollagen
adalimumab-withdrawal
glucose.
nicotine-free
meticillin-resistant
pre-docetaxel
pre-ivermectin
placebo+prednisone
glucose-corrected


#### 약-부작용 관계를 나타는 것이 아닌 레코드를 제거하면 1157 남습니다

In [26]:
drugs.shape

(1157, 2)

### 여러 약물이 하나의 레코드에 담겨있는 경우, 분리하여 입력

In [27]:
for ind, dr in drugs[(drugs.name.str.find('-')>=0) & (drugs['refined'] == '')].iterrows():
    if dr['name'].find('-to-')>=0:
        dr['refined'] = dr['name'].replace('-to-', '+')
        drugs.loc[ind] = dr

In [28]:
drugs[drugs.name.str.find('+')>=0].shape

(5, 2)

In [29]:
def separate_unique_drugs(drugs):
    additional_drugs = []

    for ind, row in drugs[drugs.refined.str.find('+')>=0].iterrows():
        processed = list(map(lambda x: x.strip(), row['refined'].split('+')))

        if len(processed)>0:
            drugs.loc[ind].refined = processed[0]
            for pro in processed[1:]:

                additional_drugs.append({'name':row['name'], 'refined':pro})

    for ind, row in drugs[drugs.refined.str.find(',')>=0].iterrows():
        processed = list(map(lambda x: x.strip(), row['refined'].split(',')))

        if len(processed)>0:
            drugs.loc[ind].refined = processed[0]
            for pro in processed[1:]:

                additional_drugs.append({'name':row['name'], 'refined':pro})


    drugs = drugs.append(pd.DataFrame(additional_drugs))
    drugs.drop_duplicates(inplace=True)
    return drugs

drugs = separate_unique_drugs(drugs)

#### 식별 가능한 약물명을 나누고 나면 1240개가 됩니다.

In [30]:
drugs.shape

(1240, 2)

In [31]:
drugs[drugs.name.str.find('taxane-cyclophosphamide')>=0]

,name,refined
105,taxane-cyclophosphamide,taxane
286,mono-taxane-cyclophosphamide,taxane
11,taxane-cyclophosphamide,cyclophosphamide
23,mono-taxane-cyclophosphamide,cyclophosphamide


### 정제후 다시 한번 dictionary와 매칭

In [32]:
apply_dictionary()

### 앞 뒤의 -, . 제거

In [33]:
remove_list = ['.', '-', ':', '+', '–', ';', '%']
def preprocess(name):
    if name[0] in remove_list:
        name = name[1:len(name)]
    
    if name[len(name)-1] in remove_list:
        name = name[0:len(name)-1]
    return name
    
drugs.name = drugs.name.map(lambda x: preprocess(x))

In [34]:
drugs = replace_with_manual_curation(drugs)
drugs = separate_unique_drugs(drugs)

/home/grace/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [35]:
drugs[(drugs['refined'] == '')]

,name,refined
6,non-naproxen,
15,deutetrabenazine,
23,isoniazidthiacetazone,
37,heparinglycoprotein,
72,amphotericin bparomomycin,
76,spartesunate,
80,prothrombin,
86,oxaliplatin–fluoropyrimidine–bevacizumab,
107,vandetanibdocetaxel,
115,tetracyclines,


In [36]:
for ind, dr in drugs[(drugs['refined'] == '')].iterrows():
    found_in_excel = replace_excel[replace_excel.key.str.find(dr['name'])>=0]
    if len(found_in_excel)>0:
        print(dr['name'])
        print(found_in_excel)

peginterferon
                       key           replace_with
280  peginterferon alfa-2b  peginterferon alfa-2b


In [37]:
drugs[drugs['refined'] == '']

,name,refined
6,non-naproxen,
15,deutetrabenazine,
23,isoniazidthiacetazone,
37,heparinglycoprotein,
72,amphotericin bparomomycin,
76,spartesunate,
80,prothrombin,
86,oxaliplatin–fluoropyrimidine–bevacizumab,
107,vandetanibdocetaxel,
115,tetracyclines,


#### 추가 전처리 필요없는 약물들은  refined로 입력

In [38]:
# spe_chars = dict((re.escape(str(k)), ' ') for k in remove_list)
# sep_pattern = re.compile("|".join(spe_chars.keys()))
# int_pattern = re.compile('[0-9]')

# for ind, row in drugs[drugs['refined'] == ''].iterrows():
#     refined = sep_pattern.sub(lambda m: spe_chars[re.escape(m.group(0))], row['name'])
#     int_refined = int_pattern.search(row['name'])
    
#     if refined.strip() == '':
#         drugs.drop(ind, inplace=True)
#     elif row['name'].strip() == refined.strip() and int_refined is None:
#         drugs.loc[ind]['refined'] = refined
#         print(row['name'])
#         print(drugs.loc[ind]['refined'])
#         print(row)
#     print('===============================')

## 정체된 목록입니다.

In [39]:
drugs.loc[drugs.refined.str.strip()!=''].to_csv('./refined.csv', sep=',')

### 추가 확인 요청 데이터

In [40]:
drugs[drugs['refined'] == '']

,name,refined
6,non-naproxen,
15,deutetrabenazine,
23,isoniazidthiacetazone,
37,heparinglycoprotein,
72,amphotericin bparomomycin,
76,spartesunate,
80,prothrombin,
86,oxaliplatin–fluoropyrimidine–bevacizumab,
107,vandetanibdocetaxel,
115,tetracyclines,


In [43]:
drugs[drugs['refined'] == ''].to_csv('./need_to_refined_0225.csv', sep=',')

In [42]:
drugs.describe()

,name,refined
count,1184,1184
unique,1101,941
top,rituximab-chop,
freq,5,77
